In [1]:
import pandas as pd

In [2]:
file_path = "./safety/features/part-0000{}-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv"
features_arr = []
for i in range(10):
    features_arr.append(pd.read_csv(file_path.format(i)))

In [18]:
label = pd.read_csv("./safety/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv")

In [22]:
label['bookingID'] = label['bookingID'].astype(str)

In [26]:
features_arr[0].head(5)

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
0,1202590843006,3.000,353.0,1.228867,8.900100,3.986968,0.008221,0.002269,-0.009966,1362.0,0.000000
1,274877907034,9.293,17.0,0.032775,8.659933,4.737300,0.024629,0.004028,-0.010858,257.0,0.190000
2,884763263056,3.000,189.0,1.139675,9.545974,1.951334,-0.006899,-0.015080,0.001122,973.0,0.667059
3,1073741824054,3.900,126.0,3.871543,10.386364,-0.136474,0.001344,-0.339601,-0.017956,902.0,7.913285
4,1056561954943,3.900,50.0,-0.112882,10.550960,-1.560110,0.130568,-0.061697,0.161530,820.0,20.419409


In [10]:
for df in features_arr:
    df['bookingID'] = df['bookingID'].astype(str)

In [73]:
all_features = pd.concat(features_arr)

In [74]:
all_features.shape

(16135561, 11)

In [78]:
all_features.columns

Index(['bookingID', 'Accuracy', 'Bearing', 'acceleration_x', 'acceleration_y',
       'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z', 'second', 'Speed'],
      dtype='object')

In [79]:
grouped_by_booking_id = all_features.sort_values("second").groupby('bookingID', sort=False)